In [1]:
import os 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,models

In [3]:
class BrainTumorDataset(Dataset):
    def __init__(self, data_dir, classes, transform=None):
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform

        # collect image paths and labels
        self.image_paths = []
        self.labels = []
        for label, class_name in enumerate(classes):
            class_dir = os.path.join(data_dir, class_name)
            for file in os.listdir(class_dir):
                if file.lower().endswith((".jpg", ".png", ".jpeg")):
                    self.image_paths.append(os.path.join(class_dir, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
classes=["glioma","meningioma","pituitary","notumor"]

data_dir="./brain_dataset"
train_dir=os.path.join(data_dir,"Training")
test_dir=os.path.join(data_dir,"Testing")



In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

train_ds = BrainTumorDataset(train_dir, classes, transform=train_transform)
test_ds = BrainTumorDataset(test_dir, classes, transform=val_transform)

num_workers = os.cpu_count() 

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=num_workers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
model=models.resnet18(pretrained=True)
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,len(classes))
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-4)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=3)


c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
best_val_loss=float("inf")
num_epochs=10

In [8]:
from tqdm import tqdm  # safer in Jupyter/Colab
import torch
from torch.utils.data import random_split, DataLoader

# ===================================================
# Auto-split Training dataset into Train + Validation
# ===================================================

# Assuming `train_ds` is created from Training/ directory earlier
val_size = int(0.2 * len(train_ds))  # 20% for validation
train_size = len(train_ds) - val_size

train_ds, val_ds = random_split(train_ds, [train_size, val_size])

# DataLoaders (Windows-safe num_workers)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)

# ===================================================
# Safety Flags for Stable GPU Training
# ===================================================
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

# ===================================================
# Training Loop
# ===================================================
best_val_loss = float("inf")
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", leave=False)

    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

        train_pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{(running_corrects.double() / ((len(train_pbar)*train_loader.batch_size))).item():.3f}"
        })

    epoch_loss = running_loss / len(train_ds)
    epoch_acc = running_corrects.double() / len(train_ds)

    # ===================================================
    # Validation Step
    # ===================================================
    model.eval()
    val_loss, val_corrects = 0.0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating", unit="batch", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    val_loss = val_loss / len(val_ds)
    val_acc = val_corrects.double() / len(val_ds)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss={epoch_loss:.4f} Acc={epoch_acc:.4f} | "
          f"Val Loss={val_loss:.4f} Acc={val_acc:.4f}")

    scheduler.step(val_loss)

    # ===================================================
    # Save Best Model
    # ===================================================
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_brain_tumor_resnet18.pth")
        print("✅ Saved best model.\n")


Epoch 1/10 | Train Loss=0.2728 Acc=0.8980 | Val Loss=0.0960 Acc=0.9641
✅ Saved best model.



Epoch 2/10 | Train Loss=0.1027 Acc=0.9665 | Val Loss=0.0707 Acc=0.9834
✅ Saved best model.



Epoch 3/10 | Train Loss=0.0619 Acc=0.9799 | Val Loss=0.0551 Acc=0.9834
✅ Saved best model.



Epoch 4/10 | Train Loss=0.0466 Acc=0.9851 | Val Loss=0.0602 Acc=0.9764


Epoch 5/10 | Train Loss=0.0412 Acc=0.9858 | Val Loss=0.0620 Acc=0.9807


Epoch 6/10 | Train Loss=0.0197 Acc=0.9945 | Val Loss=0.0472 Acc=0.9869
✅ Saved best model.



Epoch 7/10 | Train Loss=0.0227 Acc=0.9926 | Val Loss=0.0541 Acc=0.9816


Epoch 8/10 | Train Loss=0.0186 Acc=0.9954 | Val Loss=0.0534 Acc=0.9860


Epoch 9/10 | Train Loss=0.0218 Acc=0.9939 | Val Loss=0.0572 Acc=0.9834


Epoch 10/10 | Train Loss=0.0079 Acc=0.9989 | Val Loss=0.0344 Acc=0.9930
✅ Saved best model.



In [ ]:
model.load_state_dict(torch.load("best_brain_tumor_resnet18.pth"))
model.eval()
test_corrects=0
total=0

with torch.no_grad():
    for inputs,labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        _,preds=torch.max(outputs,1)
        test_corrects+=torch.sum(preds==labels.data)
        total+=labels.size(0)

test_acc=test_corrects.double()/total
print("Test accuracy:", test_acc.item())

C:\Users\vedan\AppData\Local\Temp\ipykernel_26556\2316853588.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_brain_tumor_resnet18.

In [ ]:
# # brain_tumor_train.ipynb

# import os
# import numpy as np
# from PIL import Image
# import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset, random_split
# from torchvision import transforms, models

# # ==========================================
# # Custom Dataset Class
# # ==========================================

# class BrainTumorDataset(Dataset):
#     def __init__(self, root_dir, classes, transform=None):
#         """
#         Custom Dataset for Brain Tumor MRI images.
#         Args:
#             root_dir (str): path to folder containing class subfolders.
#             classes (list): list of class names.
#             transform (torchvision.transforms): preprocessing transforms.
#         """
#         self.root_dir = root_dir
#         self.classes = classes
#         self.transform = transform
#         self.image_paths = []
#         self.labels = []

#         for idx, cls in enumerate(classes):
#             cls_folder = os.path.join(root_dir, cls)
#             if not os.path.isdir(cls_folder):
#                 continue
#             for fname in os.listdir(cls_folder):
#                 if fname.lower().endswith((".png", ".jpg", ".jpeg")):
#                     self.image_paths.append(os.path.join(cls_folder, fname))
#                     self.labels.append(idx)

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         label = self.labels[idx]
#         image = Image.open(img_path).convert("RGB")

#         if self.transform:
#             image = self.transform(image)
#         return image, label


# # ==========================================
# # Paths & Config
# # ==========================================

# # Your dataset structure
# # brain_dataset/
# # ├── Training/
# # └── Testing/

# classes = ["glioma", "meningioma", "pituitary", "notumor"]

# data_dir = r"C:\Users\vedan\MediScan\brain_dataset"
# train_dir = os.path.join(data_dir, "Training")
# test_dir = os.path.join(data_dir, "Testing")

# # ==========================================
# # Data Transforms
# # ==========================================

# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(15),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225]),
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225]),
# ])

# # ==========================================
# # Dataset & Auto-Split for Validation
# # ==========================================

# # Load entire training dataset first
# full_train_ds = BrainTumorDataset(train_dir, classes, transform=train_transform)

# # Split into train (80%) and validation (20%)
# val_size = int(0.2 * len(full_train_ds))
# train_size = len(full_train_ds) - val_size
# train_ds, val_ds = random_split(full_train_ds, [train_size, val_size])

# # Load test dataset separately
# test_ds = BrainTumorDataset(test_dir, classes, transform=val_transform)

# print(f"✅ Dataset loaded successfully:")
# print(f"   → Training samples: {len(train_ds)}")
# print(f"   → Validation samples: {len(val_ds)}")
# print(f"   → Testing samples: {len(test_ds)}")

# # ==========================================
# # DataLoader Setup
# # ==========================================

# num_workers = max(0, (os.cpu_count() // 2) if torch.cuda.is_available() else 0)

# train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=True)
# val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)
# test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("💻 Using device:", device)

# # ==========================================
# # Model Setup (Transfer Learning)
# # ==========================================

# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(classes))
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=3)

# # ==========================================
# # Training Loop
# # ==========================================

# num_epochs = 10
# best_val_loss = float("inf")

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     running_corrects = 0

#     for inputs, labels in train_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * inputs.size(0)
#         _, preds = torch.max(outputs, 1)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_ds)
#     epoch_acc = running_corrects.double() / len(train_ds)

#     # ===== Validation =====
#     model.eval()
#     val_loss = 0.0
#     val_corrects = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item() * inputs.size(0)
#             _, preds = torch.max(outputs, 1)
#             val_corrects += torch.sum(preds == labels.data)

#     val_loss = val_loss / len(val_ds)
#     val_acc = val_corrects.double() / len(val_ds)

#     print(f"Epoch [{epoch+1}/{num_epochs}] "
#           f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} "
#           f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

#     scheduler.step(val_loss)

#     # Save best model
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), "best_brain_tumor_resnet18.pth")
#         print("💾 Saved best model.\n")

# # ==========================================
# # Testing
# # ==========================================

# model.load_state_dict(torch.load("best_brain_tumor_resnet18.pth", map_location=device))
# model.eval()

# test_corrects = 0
# total = 0
# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         test_corrects += torch.sum(preds == labels.data)
#         total += labels.size(0)

# test_acc = test_corrects.double() / total
# print(f"✅ Test Accuracy: {test_acc.item():.4f}")

# # ==========================================
# # Visualization (Optional)
# # ==========================================

# def visualize_predictions(model, ds, num=6):
#     model.eval()
#     fig = plt.figure(figsize=(12, 6))
#     for i in range(num):
#         img, label = ds[i]
#         input_img = img.unsqueeze(0).to(device)
#         with torch.no_grad():
#             output = model(input_img)
#             _, pred = torch.max(output, 1)
#         ax = fig.add_subplot(1, num, i+1)
#         npimg = img.permute(1, 2, 0).cpu().numpy()
#         npimg = (npimg * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]))
#         npimg = np.clip(npimg, 0, 1)
#         ax.imshow(npimg)
#         ax.set_title(f"GT: {classes[label]}\nPred: {classes[pred.item()]}")
#         ax.axis("off")
#     plt.tight_layout()
#     plt.show()

# visualize_predictions(model, test_ds, num=6)


✅ Dataset loaded successfully:
   → Training samples: 4570
   → Validation samples: 1142
   → Testing samples: 1311
💻 Using device: cuda


c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
